In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers. imoprt Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.layers.experimental import preprocessing
import time
import pandas as pd
from shutil import copy2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
train_directory = '...'
validation_directory = '...'
world_files = '...'
dst = '...'
INPUT_DIM = ...

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_directory,
    image_size = (INPUT_DIM, INPUT_DIM),
    batch_size = 4096)


In [ ]:
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    validation_directory,
    image_size = (INPUT_DIM, INPUT_DIM),
    batch_size = 4096)


In [ ]:
base_model = tf.keras.application.VGG16(input_shape=(64,64,3),
                                       include_top=False,
                                       weights='imagenet')
model = base_model
model.summary()

In [ ]:
feature_maps = [layer.output for layer in model.layers]
activation_model = Model(inputs = model.input, outputs=feature_maps)
activation_model.summary()

In [ ]:
test_image = '...'
image = tf.keras.preprocessing.image.load_img(test_image, color_mode='rgb', target_size=(224,224))
image = tf.convert_to_tensor(tf.keras.preprocessing.image.img_to_array(image))/255.
activations = activation_model.predict(tf.expand_dims(image,axis=0))

In [ ]:
image_size=5
num_of_channels=4

for i in range(1,len(activations)):
    print('feature map number--->',i)
    
    axes=[]
    fig=plt.figure()
    
    fig.set_figheight(image_size)
    fig.set_figwidth(image_size)
    
    for channel in range(num_of_channels):
        axes.append(fig.add_subplot(int(np.sqrt(num_of_channels)), int(np.sqrt(num_of_channels)), channel+1))
        subplot_title=("Channel"+str(channel))
        axes[-1].set_title(subplot_title)
        plt.imshow(action[i][0,:,:,channel])
    fig.tight_layout()
    plt.show()

In [ ]:
backbone = MobileNetV2(input_shape=(224,224,3),include_top=False,weights='imagenet')
backbone.summary()

In [ ]:
backbone.trainable=False

In [ ]:
inputs = Input(shape=(64,64,3))
x= backbone(inputs)
x = GlobalAveragePooling2D(x)
x = Dense(1000)(x)
outputs = Dense(200, activation='softmax')(x)
model = Model(inputs,outputs)

In [ ]:
model.summary()

In [ ]:
logdir = '...'
file_writer=tf.summary.create_file_writer(logdir='/params')
file_writer.set_as_default()

In [ ]:
class LossCallback(tf.kears.callbacks.Callback):
    def on_train_batch_end(self,batch,logs=None):
        print('for batch {},loss is {:.2f}'.format(batch,logs['loss']))
        
    def on_epoch_end(self,epoch,logs=None):
        print('epoch {} loss and metrics are {:.2f}'.format(epoch,logs))

In [ ]:
def lrscheduler(epoch,lr):
    lr_init=1e-5
    if epoch<5:
        lr=lr_init
    elif epoch>=5 or epoch<10:
        lr=lr_init*10
    else:
        lr=lr*0.01
    tf.summary.scalar('learning rate', data=lr,step=epoch)
    return lr

In [ ]:
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lrscheduler)

In [ ]:
lr_plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',factor=0.5,patience=3,min_lr=1e-6)

In [ ]:
lr_scheduler = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,mode='auto')

In [ ]:
checkpoint=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
    )

In [ ]:
t_callback=tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
model.compile(optimizer=Adam(lr=1e-3),
             loss=SparseCategoricalCrossentropy(),
             metrics=[SparseCategoricalAccuracy(),SparseTopKCategoricalAccuracy(k=5)])

In [ ]:
NUM_EPOCHS = 15
history=model.fit(train_dataset,validation_data=val_dataset,epochs=NUM_EPOCHS,callbacks=[...])

In [ ]:
%tensorboard --logdir imagenet_logs/scalars

In [ ]:

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])


plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])


plt.plot(history.history['sparse_top_k_categorical_accuracy'])
plt.plot(history.history['val_sparse_top_k_categorical_accuracy'])

plt.legend(['train', 'val',
            'sparse_categorical_accuracy','val_sparse_categorical_accuracy',
            'val_sparse_top_k_categorical_accuracy'], loc='upper left')
plt.show()